In [1]:
#importing libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
df_train = pd.read_csv("..\..\Project_DATA\HealthCareCat_Train.csv")
df_dev = pd.read_csv("..\..\Project_DATA\HealthCareCat_Dev.csv")
df_test = pd.read_csv("..\..\Project_DATA\HealthCareCat_test.csv")

In [3]:
allFeatures = pd.read_csv("..\..\Project_DATA\FeatureSelection_results.csv")
selectedRFFeatures = allFeatures[allFeatures.RF == 1]

featuresToDrop = np.setdiff1d(allFeatures.Feature,selectedRFFeatures.Feature)

df_train = df_train.drop(featuresToDrop, axis=1)
df_dev = df_dev.drop(featuresToDrop, axis=1)
df_test = df_test.drop(featuresToDrop, axis=1)


In [4]:
### df[df.Spending_Cat == 1] -- 125309
### df[df.Spending_Cat == 2] -- 4719  --   x 25
### df[df.Spending_Cat == 3] -- 1075  --   x 115
### df[df.Spending_Cat == 4] -- 170   --   x 736

In [5]:
### 
X_train = df_train.drop("Spending_Cat",1)   #Feature Matrix
y_train = df_train["Spending_Cat"]          #Target Variable

X_dev = df_dev.drop("Spending_Cat",1)   #Feature Matrix
y_dev = df_dev["Spending_Cat"]          #Target Variable

X_test = df_test.drop("Spending_Cat",1)   #Feature Matrix
y_test = df_test["Spending_Cat"]          #Target Variable

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)

RandomForestClassifier(random_state=0)

In [42]:
res = []

In [37]:
yhat_train = rf.predict(X_train)
pd.crosstab(y_train,yhat_train, rownames=['Actual'], colnames=['Predicted'])

Predicted,1,2,3,4
Actual,,,,
1,80277,8,0,0
2,1,79983,0,0
3,0,0,80216,0
4,0,0,0,80263


In [38]:
yhat_dev = rf.predict(X_dev)
pd.crosstab(y_dev,yhat_dev, rownames=['Actual'], colnames=['Predicted'])

Predicted,1,2,3,4
Actual,,,,
1,19076,868,5,0
2,300,19950,0,0
3,2,2,20014,0
4,0,0,0,19971


In [39]:
yhat_test = rf.predict(X_test)
pd.crosstab(y_test,yhat_test, rownames=['Actual'], colnames=['Predicted'])

Predicted,1,2,3,4
Actual,,,,
1,24659,386,2,0
2,130,805,1,0
3,21,24,190,0
4,1,4,2,30


In [43]:
res.append(
            {'model':'RF - Initial',
               'accuracy-Train':rf.score(X_train,y_train),
               'accuracy-Dev':rf.score(X_dev,y_dev),
               'accuracy-Test':rf.score(X_test,y_test),
               'auc-Train': roc_auc_score(y_train, rf.predict_proba(X_train), multi_class='ovr'),
               'auc-Dev': roc_auc_score(y_dev, rf.predict_proba(X_dev), multi_class='ovr'),
               'auc-Test': roc_auc_score(y_test, rf.predict_proba(X_test), multi_class='ovr')
            }
          )
res

[{'model': 'RF - Initial',
  'accuracy-Train': 0.9999719405888735,
  'accuracy-Dev': 0.9853219933157081,
  'accuracy-Test': 0.9782517615692249,
  'auc-Train': 0.9999995868323275,
  'auc-Dev': 0.9995041675005393,
  'auc-Test': 0.9834554413804016}]

#### Random Hyperparameter Grid

In [9]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1200, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 6)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 450, 700, 950, 1200], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 30, 50, 70, 90, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:

rf1 = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf1, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=3, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 89.0min
